# RAG With llama-index  + Milvus + Qwen - Part 2

References

- https://studio.nebius.com/
- https://docs.llamaindex.ai/en/stable/examples/vector_stores/MilvusIndexDemo/
- https://docs.llamaindex.ai/en/stable/api_reference/storage/vector_store/milvus/?h=milvusvectorstore#llama_index.vector_stores.milvus.MilvusVectorStore

- 1) проверьте файл .env
- 2) kernel - studio1

## Step-1: Configuration

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

if os.getenv('NEBIUS_API_KEY'):
    print ("✅ Found NEBIUS_API_KEY in environment, using it")
else:
    raise ValueError("❌ NEBIUS_API_KEY not found in environment. Please set it in .env file before running this script.")

✅ Found NEBIUS_API_KEY in environment, using it


## Step-2: Setup Embedding Model

We have a choice of local embedding model (fast) or running it on the cloud

If running locally:
- choose smaller models
- less accuracy but faster

If running on the cloud
- We can run large models (billions of params)

In [2]:
import os
from llama_index.core import Settings

# Option 1: Running embedding models on Nebius cloud
#from llama_index.embeddings.nebius import NebiusEmbedding
#EMBEDDING_MODEL = 'Qwen/Qwen3-Embedding-8B'  # 8B params
#EMBEDDING_LENGTH = 4096  # Length of the embedding vector
#Settings.embed_model = NebiusEmbedding(
#                        model_name=EMBEDDING_MODEL,
#                        embed_batch_size=50,  # Batch size for embedding (default is 10)
#                        api_key=os.getenv("NEBIUS_API_KEY") # if not specfified here, it will get taken from env variable
#                       )

# Option 2: Running embedding models locally
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
Settings.embed_model = HuggingFaceEmbedding(
    # model_name = 'sentence-transformers/all-MiniLM-L6-v2' # 23 M params
    model_name = 'BAAI/bge-small-en-v1.5'  # 33M params
    # model_name = 'Qwen/Qwen3-Embedding-0.6B'  # 600M params
    # model_name = 'BAAI/bge-en-icl'  # 7B params
    #model_name = 'intfloat/multilingual-e5-large-instruct'  # 560M params
)

/home/dasha/.conda/envs/studio-1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step-3: Connect to Milvus

In [4]:
from pymilvus import MilvusClient

DB_URI = './rag.db'  # For embedded instance
COLLECTION_NAME = 'rag'

milvus_client = MilvusClient(DB_URI)
print ("✅ Connected to Milvus instance: ", DB_URI)


✅ Connected to Milvus instance:  ./rag.db


In [5]:
# connect to vector db
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.milvus import MilvusVectorStore

EMBEDDING_LENGTH = 384
vector_store = MilvusVectorStore(
    uri = DB_URI ,
    dim = EMBEDDING_LENGTH ,
    collection_name = COLLECTION_NAME,
    #overwrite=False
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

print ("✅ Connected Llama-index to Milvus instance: ", DB_URI )

✅ Connected Llama-index to Milvus instance:  ./rag.db


## Step-4: Load Document Index from DB

In [6]:
%%time

from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, storage_context=storage_context)

print ("✅ Loaded index from vector db:", DB_URI )

✅ Loaded index from vector db: ./rag.db
CPU times: user 139 ms, sys: 16 ms, total: 155 ms
Wall time: 154 ms


## Step-5: Setup LLM

In [ ]:
#from llama_index.llms.nebius import NebiusLLM
#from llama_index.core import Settings
#
#Settings.llm = NebiusLLM(
#                #model='openai/gpt-oss-120b',
#                #model='Qwen/Qwen3-30B-A3B',
#                model='deepseek-ai/DeepSeek-R1-0528',
#                api_key=os.getenv("NEBIUS_API_KEY") # if not specfified, it will get taken from env variable
#    )
# no access 403

In [ ]:
#from llama_index.llms.ollama import Ollama
#from llama_index.core import Settings
#
#Settings.llm = Ollama(
#    model="llama3.1",  #"mistral", "qwen2.5:7b"
#    request_timeout=120.0
#)
#
#
#model requires more system memory (29.7 GiB) than is available (14.1 GiB) (status code: 500)

In [7]:
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import Settings
import sacremoses

Settings.llm = HuggingFaceLLM(
    model_name="microsoft/BioGPT-Large",
    tokenizer_name="microsoft/BioGPT-Large",
    device_map="auto",
    max_new_tokens=512,
    context_window=2048,
    generate_kwargs={
        "temperature": 0.3,
        "do_sample": True,
        "top_p": 0.9,
        "repetition_penalty": 1.1
    }
)

In [ ]:
# это чот пиздец долго думает
# from llama_index.llms.huggingface import HuggingFaceLLM
#from llama_index.core import Settings
#
#
#Settings.llm = HuggingFaceLLM(
#    model_name="mistralai/Mistral-7B-Instruct-v0.3",
#    tokenizer_name="mistralai/Mistral-7B-Instruct-v0.3",
#    device_map="auto",
#    max_new_tokens=512,
#    context_window=4096,
#    generate_kwargs={"temperature": 0.7, "do_sample": True}
#)

## Step-6: Query

In [8]:
query_engine = index.as_query_engine()
res = query_engine.query("What was Uber's revenue for 2020?")
print("Response:", res)

Response: Empty Response


## Making sure the model uses context

Let's ask a generic factual question "When was the moon landing".

Now the model should know this generic factual answer.

But since we are querying documents, we want to the model to find answers from within the documents.

It should come back with something like "provided context does not have information about moon landing"

In [9]:
query_engine = index.as_query_engine()
res = query_engine.query("When was the moon landing?")
print(res)

Empty Response
